# Lab 8: PID Control

In previous lab sheets, you have:

- Used millis() and micros() to take measurements of time, and used these to schedule different tasks at different times.
- Controlled the speed of the two ROMI motors with `analogWrite()`, and direction of the two Romi motors using `digitalWrite()`.
- Achieved a count of encoder clicks using a Pin Change Interrupt ISR.
- Determined an estimate of the speed of rotation of your Romi's wheels.

In this lab sheet, we will apply a **closed loop** control approach known as **PID** (Proportional, Integral, Derivative). A PID controller takes a **demand** as an input, compares it to a **measurement**, and then produces an **output signal**.  We would want to use a PID controller in order that a system can regulate itself.  For example, you might want your Romi to drive at the same velocity regardless of whether there is more or less resistance under the wheels (e.g., driving on mud 
versus a table).  

In this coursework assessment, the PID routine will give you an advantage in the final objective of `Go Home` - the last 35 marks toward a distinction.  In order to get home with accuracy and precision, you need your wheel encoders to produce the least amount of error.  PID control should allow your robot to drive smoothly, reducing error.  When your robot will travel back to where it started, it will not have a convenient line to follow (unless you use this as a strategy).

In robotics, we generally want robust, reliable and efficient autonomous systems - systems which can adapt and generalise to increasingly complex circumstances.  It is less desirable to have a robot which must operate in very fixed and stable environment.  A feedback controller is a step toward an adaptive autonomous system.  By following this labsheet, your Romi will be able to autonomously adjust the left and right wheel speed smoothly to follow the line and drive in a straight line.  Later on, you could use kinematics so that your Romi can drive in a straight line by maintaining a desired calculated direction.

The below diagram provides an overview of a PID controller:

<img src="https://github.com/paulodowd/EMATM0054_20_21/blob/master/images/PID_Overview.png?raw=true"/>

In this labsheet, we are going to use a PID controller to operate the motors and use the encoder counts as the feedback control  mechanism.  Referring to the above diagram:
- the **demand** will be a desired encoder count (or encoder count per second).
- the **measurement** will be the actual encoder count from the Romi.
- the **error signal** is the difference between **measurement** and **demand**.
- the **output signal** is a PWM waveform sent to the motor via _analogWrite()_.


This Labsheet will cover a **Speed** controller - using PID to maintain a rotation velocity for a wheel.

It is likely that you will use a PID controller in many places to control your Romi, so it makes good sense to use a class again.  You will find a template PID class (pid.h) in the provided Code Stub.






<br>
<br>
<br>
<br>
<br>

# Concerning PID Controllers:

A PID controller can be used to control all sorts of systems.  For instance, the temperature of a room: 
- the **measurement** would be the sensed room temperature.
- the **output** would be a heating element, to effect the temperature of the room.
- the **demand** input would be the user temperature control on the wall.

Therefore the operation of a PID controller can be described in natural language as:
- **measuring** the temperature of the room
- attempting to match the measurement to the user **demand** (e.g., what is the difference, the **error**)
- adjusting the **output** heating element _appropriately_ to be hotter or cooler, to raise or lower the temperature of the room.

It is important to note that the measurement, output and demand can each be of different units and of different scales:
- The user demand might be in Celcius in 1 degree increments in the range [5 : 30].
- The measurement might be a 12-bit digital value from a linear temperature sensor
- The output might be a high electical-current to drive a non-linear industrial heater working in kilo-Watts.  

Therefore, a PID controller must map a **relationship** between demand, output and measurement (e.g., room gets too hot -> heater must get cooler).  Furthermore, there is a necessary **transformation** that must occur between:
- making a comparison between measurement and demand to determine the **error signal**
- transforming the error signal to correct the **output**.

In the above example of maintaining the temperature of a room, we would need to transform a **small error signal** (between a temperature sensor and user demand), to a **large voltage** for a kilo-Watt heater.  For controlling the motor on the Romi, we start with an **encoder count** (`unsigned long`), and we must transform this into an **analogWrite()** (`[0:255]`).  Every system will:
- have a different processing of demand, measurement, error signal and output.
- the relationship between demand and output will be different. 
- the transformation from error signal to output will be different.
- the actual physical system will behave differently.

The advantage of being 'unit agnostic' (different units on input and output) is that a PID controller can be used to provide closed-loop control between any input signal and output signal, so long as their is a strong causal relationship.  Therefore, you may later use another PID controller to minimise the error of other system state variables.  It is likely that you will use the output of one PID controller as the demand signal for another PID controller (called, nested control).

Whilst it is good that a PID controller can be 'unit agnostic', there is a clear drawback.  Every PID controller requires careful implementation and calibration.  PID controllers have a reputation for being annoying to work with because there are plenty of parts to go wrong.  When they go wrong, they tend to oscillate or exhibit chaotic, hard to understand behaviour.  Therefore, it is important to only take small steps and make small changes.  It is important to approach implementing a PID controller in a methodical way - a way which builds up complexity and removes as many errors and oversights as possible.

### Step 1: Check your Error Signal

The first task is to determine and check an error signal (demand vs measurement):
 - Check the error signal looks sensible, for example:
     - what are you measuring, what is a sensible range of values?
     - does it go both positive and negative?
     - does positive and negative make sense (e.g. correlates to forward and backward?)
     - is the error signal stable?

### Step 2: Tuning your PID

A PID controller is going to take your **error signal** and transform it into a meaningful **output signal** to drive your system.  To create this transformation, a PID controller must be **calibrated**, or **tuned**.   You may find tuning a PID controller difficult.  You should find a lecture to help guide you in this process.  You may arrive at a controller which does not use all terms (P, I and D) - such as a PD controller, or just a P controller - it depends on your task requirements.

The transformation of error signal to output signal is created by setting the **P, I and D gains**.  It is best to work with one gain at a time.  Simply put:
- **P-gain** multiplies (or divides) the error signal to produce the output signal, and it is the first gain you should set.  It is the most straight forward mapping of error signal to output signal.  
    - You can think  of the P-Gain as an aggressive response (a shunt) to try to "jump" the output signal to correct for error.  When the error decreases, so will the effect of the P-Gain.  If the error is 0, your P-Gain will multiple out to 0 as well.  Therefore, P-Gain has less influence on your system in low-error conditions.
    - It is unlikely that you will find a perfect p-gain value.  Generally, it is better for your system to undershoot, rather than overshoot.  
        - Overshoot can quickly become oscillation.  Trying to fix this with D-Gain will only create worse oscillations.  
        - Undershoot results in a steady state error, which can be fixed with I-Gain.
- **D-gain** compensates the P-gain, by producing an effect based on the rate-of-change.  You may notice your P-gain causes overshoot, and the D-gain can help to counteract this.
    - You can think of the P-Gain as acting like a dampening spring which resists sudden movement.
    - Don't use D-Gain to try to fix persistent oscillation, this is not it's purpose.  It will only cause more oscillation.
    - Only use D-Gain to attempt to "smooth" the response of your system under significant changes in the demand.
- **I-gain** adds to the output signal when an error signal is sustained over time.  It effectively 'accumulates' error if the target has not been reached quickly enough.  The I-gain is normally the most sensitive, and therefore has a very small gain-value.  It is best to experiment with the I-gain last.  
    - You can think of the I-Gain as a gentle persistent force to correct for an error signal that has not been fixed by the P-Gain.  
    - However, when I-Gain goes wrong, it is the opposite of gentle!

To tune your PID controller, you need to actually work with the physical system to: 
- understand how it behaves (often, the behaviour, or relationship, is non-linear).
- understand the characteristics of measurement, demand, error signal and output.

Completing the tasks in the previous labsheets should have helped you to gain an understanding some of the complexity of the performance of your Romi.  For instance, you might already be aware that your left motor is slower than your right motor (or vice-versa) when given the same power.  Each Romi will be slightly different.  

### Important Questions to Consider

You should approach tuning your PID controller with patience and a step-by-step process.  Work on one motor at a time.  Use the Serial Plotter to inspect the various calculations.  Tuning your PID controllers well will really help the overall performance of your system.

When implementing a PID controller we must first ask:
- Is there a relationship between error signal (demand vs measurement) and output?

If the answer is yes, then your system may be able to regulate itself.   However, we must then ask or investigate the following question:
- What are the characteristics of demand, measurement, error signal, and output?
    - Think about this question carefully, sketch down some graphs you'd expect to see.
    - Make sure you understand what a given error signal should be transformed into (by a gain value) to produce a useful output signal.
        - For example, if you observe your error signal on the Serial Monitor (or plotter), work out what P-Gain value would produce a useful output (analogWrite needs a value between 0 : 255 ).
    - Make sure you report useful information back using `Serial.print()`, and make sure you can understand what you are observing.
    - Work with one gain at a time, and make sure the relationship is as you would expect.
    - You can get strange, half-working performance if your error signal is inverted.


# PID Class: pid.h

Take a look at the code in the "pid.h" tab. In particular, you should find:
- The definition of the class (highlighted by the keyword <i>class</i>)
    - Some <i>public </i> functions
    - Some <i> private </i> functions and variables
- Definitions and code for both the private and public functions
    
Take a moment to try to understand the high level flow of the code. If you are unsure, ask a TA (or other student) to help. 

Most of the functions in the class have already been implemented, with the exception of the <b>update</b> function. Your job for today is to first implement this function and then investigate the performance of the PID controller. If we look at the update function, you will find the following code:

```C++
/*
 * This is the update function. 
 * This function should be called repeatedly. 
 * It takes a measurement of a particular variable (ex. Position, speed, heading) and a desired value for that quantity as input
 * It returns an output; this can be sent directly to the motors, 
 * combined with other control outputs
 * or sent as input to another controller
 */
float PID_c::update(float demand, float measurement) {
  //Calculate how much time (in milliseconds) has passed since the last update call
  long time_now = millis();
  int time_delta = time_now - last_millis;
  last_millis = time_now;

  /*
   * ================================
   * Your code goes implementation of a PID controller should go here
   * ================================
   */

  //This represents the error term
  // Decide what your error signal is (demand vs measurement)
  float error;
  error = ????;   
  
  //This represents the error derivative
  // Calculate the change in your error between update()
  float error_delta;
  error_delta = ????;

  // This represents the error integral.
  // Calculate error over time.
  integral_error = ???;

  //Attenuate above error components by gain values.
  Kp_output = Kp * ????;
  Ki_output = Ki * ????;
  Kd_output = Kd * ????;

  // Add the three components to get the total output
  // Note: Check the sign of your d gain.  Check that the
  // Kd_output contribution is the opposite of any 
  // overshoot you see.
  output_signal = Kp_output + Ki_output + Kd_output;

  /*
   * ===========================
   * Code below this point should not need to be changed
   * But of course, feel free to improve / experiment :)
   */
```

&nbsp;<br>&nbsp;<br>&nbsp;<br>&nbsp;<br>

# Exercise 1: PID Speed Controller


## Task 1) Verify your Error Signal

Currently, the update() method of the pid class simply returns 0 everytime it is called, because the function has not been written. Remember that update() is returning the output signal, which should be an appropriate power value for your motor (e.g. analogWrite() [ 0:255 ], +/- for direction ).

The first thing we will do is to add the code necessary to calculate an error signal (measurement vs demand). We will then use the serial monitor to confirm the signal operates as predicted.

In order for our PID class to do anything, we must first create an instance of it, and then call the update function in our main loop. If we look inside the tab "PID_Exercise":

```C++
// Experiment with your gains slowly, one by one.
float Kp_left = 0.0; //Proportional gain 
float Kd_left = 0.0; //Derivative gain
float Ki_left = 0.0; //Integral gain
PID_c left_PID(Kp_left, Ki_left, Kd_left); // controller for left wheel
```

The above creates an instance the PID class, and sets the gain values all to 0.0.

```C++
void loop() {

  //    output_signal <----PID-- demand, measurement
  float output = left_PID.update(demand, e1_speed);
  
  Serial.print("Left wheel output is: ");
  //Serial.println(output);

  // Once you think your error signal is correct
  // And your PID response is correct
  // Send output directly to your motor
  // Be sure to handle -ve numbers and values
  // larger than 255 ( see analogWrite() )
  

  // Consider switching this delay for a millis()
  // task-schedule block.
  delay(LOOP_DELAY);
}
```

The above _loop()_ calls the update function and prints output to the serial console. 

- **Don't enable the motor power** (blue light off)
- Set a demand of 0.
- Set a P-gain (1 would report just the unmodified error signal).
- Verify that your controller produces an output signal you'd expect when you rotate the wheel backwards or forwards.


<h6> Hint:</h6> Is your output always zero? Take a look at the P gain!
<h6> Hint:</h6> Does the sign of your error signal make sense to you?  Does it relate to the direction of your robot?  There is not a right answer, but you should work with what makes sense to you, and then be consistent in your work.






&nbsp;<br>&nbsp;<br>&nbsp;<br>&nbsp;<br>

## Task 2) Tune your P-Gain
The simplest feedback controller applies a control signal proportional to the error between our actual speed (measurement) and desired position (demand):

$$U = -Kp * e = -Kp * (demand-measurement)$$

where $U$ is our control signal, $K_{p}$ is our proportional control gain and $e$ is our error signal.  

From **Labsheet 7: Time & Speed Estimation**, you should have implemented a method to estimate the speed of each wheel.  Therefore, our measurement will be this calculated value, and the demand will be an appropriate value (a similar magnitude and unit).  You could try rotating a wheel by hand and observing the reported speed measurement.  Then, aim to use that same measurement value as the demand value to test your PID controller.

**Note that, in the above expression the Kp term is negative.  The sign of the terms has more to do with your application.  Your error signal may have an inverse relationship, depending on which way you increment your encoder count with respect to the forward direction of your robot.  Generally, the PID controller is constructed by the summation the P, I and D terms ( P + I + D ), but in which case the D term has a negative gain value (because it is a compensating term).  However, we could keep all positive gains and write the expression as  P + I - D .  Therefore, make sure you understand how each term (P, I and D) is supposed to help contribute the output signal, and select the sign of your terms and gains appropriately.**

**The best way to do this, is to Plot both your demand, error and output signal to the Serial Plotter, and see if it makes sense.  Then check the rotation of your wheel is correct.**





When we tune a P-Gain for speed control, you should expect that you will be **unable** to perfectly match the demand.  This is because the P-Gain is proportional, and therefore it cannot adapt to a change of the dynamic of the systems.  What does this mean?  

Consider this: you could test and set your P-Gain whilst your Romi is lifted off the table, perhaps resting on a cup.  You could find a P-Gain that was almost perfect to match a desired measured wheel speed.  Then, when you put your Romi on the floor, the motor now experiences a different resistance (load), and your P-Gain is no longer "perfect".

The P-Gain represents a fixed relationship between the measurement and demand.  Every time you significantly change the robot or environment, you will likely need to recalibrate the P-Gain.  Therefore, attempt to tune your P-Gain to get the best match between demand and measurement in the environment you expect it to operate within.  Evaluating this aspect of a PID controller could make for an interesting experiment.

**As a rule of thumb:** it is better that your PID controller slightly undershoots the demand rather than overshoots.  When the system overshoots, it tends to oscillate.  We can fix undershoot later with the I-Gain.  A system in oscillation is not fixable.

Once you have verified your error signal and output signal are operating as you'd expect:
- Implement the motor activation
    - We have covered this in Labsheet 3.
    - It is recommended you create a class to operate your motors.  This will keep things consistent and reduce the amount of time you spend debugging.  You can use any of the example classes (pid.h, lineSensor.h, and simply rename/rewrite all the parts appropriately).
- Use the value returned by pid.update() to activate the power of your motors. 
    - Remember to change the direction pin with respect to update() returning a positive or negative result.
    - Consider what type update() returns and whether this is appropriate to send to your motors.
    
    
    
- Experimentally tune the P gain of your controller. 
    - For the tuning process, you should select a reasonable demand (not too large) and observe performance (both in terms of what the Romi does, and the PID tuning metrics we discussed in the lecture).  You want your motor to reach the demand position with:
        - a fast response
        - to bring your wheel speed as close to the demand speed as possible.
        - minimal overshoot
        - minimal oscillation
        - minimal steady state error
    - Use Serial.print() and the plotter to observe the performance of your measurement to reach demand.
    - **Hint:** If your motors are irratic, you may have your error signal and motor direction around the wrong way.  With this mistake, sometimes your wheel speed will be stable and then suddenly go to maximum speed.
    - **Hint:** It can help to implement a millis() task block to switch the demand position between a positive to negative demand value every 4 seconds.  This way, you can see the response on serial plotter again and again, albeit in different directions.
    - Be prepared to experiment in order to understand what is going on.  Try a range of demand speeds.
  
    
        



&nbsp;<br>&nbsp;<br>&nbsp;<br>&nbsp;<br>




## Task 3) Tune the I-Gain

The I-Gain is interesting, but not always necessary.  With the Romi, you may find that I-Gain is important to maintain low speeds of rotation.  Remember, when the error signal is small, so is the P-Term, and so the I-Term can help here.

The final element of the PID controller we discussed in the lecture is the integral term, which acts to reduce steady-state error. Intuitively, we need an integral term if there is some <i>springiness</i> to the dynamics of the variable we are controlling. You can think of this as the initial friction of the motor which needs to be overcome to gain smooth rotation.  The addition of this term means our full control law becomes:

$$U = -K_p * (error) - K_d * \dot{error} - K_i * \int_{0}^{T} error * dt $$

- Implement the integration error in your PID class update method.
    - Remember that we can approximate an integral with a summation
- Verify that your Romi can hold a consistent wheel speed where there measurement is just under the demand.
- Tune your I-gain
    - Start with a very small I-Gain.
    - With the motor power off, you should see the I-term accumulating error.  
    - Be sure to visualise the control response with the Serial plotter.  The I-gain can induce oscillations very easily.  
    - You can validate that the I-Term is operating by offering resistance to the rotation of the wheel, limiting its ability to match the demand.
        - Note that, if you offer resistance the I-Term will increase to overcome the resistance.  Then, when you let go, it may take a small amount of time for your I-Term to decrease again.  
        - You may also observe this if you stop your motors working (e.g. no power) whilst your PID is still computing.  The I-Term will keep accumulating error (error because the wheels are not turning).  Then when you turn on your robot, the wheels will be set to maximum power, and your Romi will drive off into the sunset.
    - Because your I-term will add to your P-term, you may need to reduce your P-gain slightly.  
    - A good test of your I-gain is to set very low demand speeds.  A proportional response would either not be enough, or cause wild overshoot and oscillation.  The I-term can nudge your motor along at low speeds, and help to mitigate the **motor power deadband**.
    - You should be able to bring your controller response up to match a demand speed very closely with the I-gain.
    - You may need to experiment with the size of the delay to get a good response from your speed controller. 




&nbsp;<br>&nbsp;<br>&nbsp;<br>&nbsp;<br>

## Task 4) Tune D-Gain (optional)

The D-Gain is interesting, but not always necessary.  You may even skip this parameter and achieve very good performance.

As we saw during the lectures, we can improve the response of our controller by adding a deriviate term which damps the response of the proportional term when the error is decreasing at a fast rate. The deriviative term tries to reduce the output signal if the error is reducing quickly.

The control law becomes:

$$U = -K_p * (error) - K_d * \dot{error}$$


  

In order to apply this control law, we need to estimate the rate at which the error is decreasing. A simple way to do this is with a finite difference approximation:

 $$\dot{error} = \frac{error(t) - error(t-1)}{dt}$$

The D-gain and D-term will be useful if your Romi changes the demand suddenly, such as going from a forward speed to a reverse speed.  When this occurs, your motor power will momentarily be in a very large error state, causing a large and inverted output signal (rapid change).  In order to tune this term, you will likely need to implement a piece of code to change your demand from positive to negative after a few seconds have elapsed.  Note that, we do not want to use the D-Gain to attempt to fix oscillation caused by continual overshoot - this will create even worse oscillation.  Ideally, the D-Gain is used to fix the system response to a dramatic change in the demand.

- Modify the PID update method to also include the derivative error term. 
    - Hint: You can use the private variable <i>last_error</i> to store the error for the next iteration of the update method.
- Tune the derivative gain of your controller. 
    - As before, use the Serial plotter to visualise the control signal.  
    - Start with a small gain and gradually increase it until you observe an effect. 
    - We would expect the contribution made by the d-term to be counter-active to the p-term.  When the p-term produces a spike (an overshoot), the d-term should produce a counter-effective spike (a spike in the opposite direction).
    - What does increasing the derivative gain to do the rise time and overshoot?

- Now adjust both the proportional and derivative terms alternately. 
    - You may be able to use a higher P-gain than before, which is compensated for by the D-gain.  Conduct your own experiments.
    - You may decide not to use a D-Gain (set to 0)
    - By including the D term, you should be able to improve the response of your controller.



&nbsp;<br>&nbsp;<br>&nbsp;<br>&nbsp;<br>

# Exercise 2: Nested Control Loops

By now you should have working a Speed Controller for one motor.  Hopefully, you took the opportunity to:
- Created a second instance of the PID class for the second motor.
- Tune the second PID instance for the second motor.

In which case, you should have a system as illustrated below:

<img src="https://github.com/paulodowd/EMATM0054_20_21/blob/master/images/TwoMotors.png?raw=true"/>

You should be able to set the same speed for both wheels and observe a reasonably straight line of travel.  Note that, both wheels are independently managing their speed or position.  

Even though you are using two speed controllers, you may observe a wobble (as depicted above) when the robots starts moving.    This is because the two PID controllers are acting independently.  This can set the robot off course.

By now you should be quite familiar with the code for your PID class, and how to use it in different applications.  We can now augment the above system to produce a nested controller system, depicted below:

<img src="https://github.com/paulodowd/EMATM0054_20_21/blob/master/images/nested.png?raw=true"/>
     
In the above, a third PID instance called "Heading PID" is instatiated to provide the demand signals to the Left PID and Right PID instances.  Take a moment to consider this. 

Previously, we have hard-coded the demand speed when developing our PID controllers.  However, it is possible for your Romi to manage the demand speeds for the left and right motor for itself.  The new **heading controller** can instruct the speed controllers with a demand speed.  One way to think about this operation, is that the Heading PID will balance (or "correct") the operation of the Left and Right motors to drive straight, or may provide bias ("unbalance" the motor speeds) to create turns.  

Used in this way, we should be able to substitute your bang-bang controller for a new, smooth and dynamic PID control system.

Good questions for you to ask yourself are:
- What is the demand, error and therefore measurement for the heading controller?
- What is a **sensible** output for the heading controller, to give to the speed controllers?

For now, use the LineSensor class to provide an **error signal** to your heading PID.  Remember that with the weighted line sensor technique, we derived a value in the range [ -1.00 : +1.00 ] to represent that the line was either to the left or right of the sensor.  In which case, what would a good **demand** value be to keep the line centered under the sensor?

Your heading controller will produce a single **output signal** which we are going to feed into the two speed controllers (left and right) as the speed demand.  What we want to see is that, when the heading controller error signal is either positive or negative, the left and right motors will change their speed to keep the line sensor centered over the line.


## Task 1)

- If you haven't already, make sure your Left and Right PID controllers are being updated at a fixed time interval, and do this using a _millis()_ task block, as per prior labsheets.  E.g., substitute the `loop_delay` from the code provided so that your _loop()_ is non-blocking.

## Task 2) 

- Save your work so far.

- Create a copy of your program and include your line sensor code class.

- Decide the following for your Heading Controller:
    - Measurement - use the measurement we developed in **Labsheet 5** for line following.  This had the range of [ -1.00 : +1.00 ] to represent where the line was under the sensor.
    - Demand - to follow the line, what should the demand be?
    - Output Signal - if your Heading Controller outputs the demand for the left/right speed controllers, what range of values should the output signal be within?

- Follow the PID tuning process for your heading controller.  Start with just a P-Gain first.  Start with a P-Gain of 0, and then slowly increase it.  You should be able to see your Romi turn to stay centered over the line.

- If either your measurement or output signal are the wrong sign, you'll have a robot which tries to avoid the line.  Sometimes it can look like it is following the line because it is "bouncing" along the outside edge.

- You may need to implement a forward bias so that your Romi drives forwards.

- Decide on how frequently to update your Heading Controller, especially with respect to how frequently you update the Left/Right PID controllers.


